In [1]:
# Dependencies
import pandas as pd
import numpy as np
import os

In [2]:
#  import csv
df = pd.read_csv('../Resources/final_data.csv')
df.head()

,county,year,violent_crime,murder,rape,robbery,aggravated_assault,property_crime,burglary,larceny_theft,...,frm_15,points_15,median_hh_income,median_hh_inc_moe,poverty_count,poverty_count_moe,poverty_rate,poverty_rate_moe,county_fips,price
0,Atlantic,2019,0,0,0,0,0,0,0,0,...,3.391731,0.475,62678,2822,29057,4251,1.6,11.3,1,196067.42
1,Bergen,2019,2,0,2,0,0,46,2,44,...,3.391731,0.475,107971,3025,52980,7662,0.8,5.7,3,494018.42
2,Burlington,2019,0,0,0,0,0,0,0,0,...,3.391731,0.475,88443,3233,24961,4374,1.0,5.7,5,238593.67
3,Camden,2019,2,0,0,0,2,44,4,38,...,3.391731,0.475,73168,2374,53641,7048,1.4,10.7,7,181980.75
4,Cape May,2019,0,0,0,0,0,0,0,0,...,3.391731,0.475,66565,4753,8853,1981,2.2,9.8,9,389294.58


In [3]:
#  drop nulls
df.dropna(how='any', inplace = True)
# df.dtypes

In [4]:
# Set features. This will also be used as x values.
X = df.drop(['county', 'county_fips'], axis=1)
y = df["county"]
print(X.shape, y.shape)

(164, 22) (164,)


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical
# Split data into training and testing groups and scale data 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train,21)
y_test_categorical = to_categorical(encoded_y_test,21)

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define the model
deep_model = Sequential()
# Add first layer
deep_model.add(Dense(units=50, activation='relu', input_dim=22))
# Add second layer (deep)
deep_model.add(Dense(units=50, activation='relu'))
# Add third layer (deep)
deep_model.add(Dense(units=50, activation='relu'))
#  Add output layer
deep_model.add(Dense(units=21, activation='softmax'))

In [7]:
#  model summary
deep_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                1150      
_________________________________________________________________
dense_1 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_2 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_3 (Dense)              (None, 21)                1071      
Total params: 7,321
Trainable params: 7,321
Non-trainable params: 0
_________________________________________________________________


In [8]:
# Compile and fit the model
deep_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# Fit the model to the training data
deep_model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2)

Epoch 1/100
4/4 - 1s - loss: 3.0483 - accuracy: 0.0488
Epoch 2/100
4/4 - 0s - loss: 3.0309 - accuracy: 0.0569
Epoch 3/100
4/4 - 0s - loss: 3.0164 - accuracy: 0.0650
Epoch 4/100
4/4 - 0s - loss: 3.0022 - accuracy: 0.1057
Epoch 5/100
4/4 - 0s - loss: 2.9862 - accuracy: 0.1382
Epoch 6/100
4/4 - 0s - loss: 2.9708 - accuracy: 0.1626
Epoch 7/100
4/4 - 0s - loss: 2.9532 - accuracy: 0.1545
Epoch 8/100
4/4 - 0s - loss: 2.9361 - accuracy: 0.1626
Epoch 9/100
4/4 - 0s - loss: 2.9143 - accuracy: 0.1707
Epoch 10/100
4/4 - 0s - loss: 2.8882 - accuracy: 0.1707
Epoch 11/100
4/4 - 0s - loss: 2.8599 - accuracy: 0.1707
Epoch 12/100
4/4 - 0s - loss: 2.8306 - accuracy: 0.1707
Epoch 13/100
4/4 - 0s - loss: 2.7961 - accuracy: 0.1789
Epoch 14/100
4/4 - 0s - loss: 2.7569 - accuracy: 0.1951
Epoch 15/100
4/4 - 0s - loss: 2.7158 - accuracy: 0.2114
Epoch 16/100
4/4 - 0s - loss: 2.6676 - accuracy: 0.2114
Epoch 17/100
4/4 - 0s - loss: 2.6139 - accuracy: 0.2520
Epoch 18/100
4/4 - 0s - loss: 2.5569 - accuracy: 0.2520
E

In [9]:
model_loss, model_accuracy = deep_model.evaluate(X_test_scaled, y_test_categorical, verbose=2)
print(f"Deep Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

2/2 - 0s - loss: 0.2636 - accuracy: 0.9024
Deep Neural Network - Loss: 0.2635520100593567, Accuracy: 0.9024389982223511


In [10]:
prediction = np.argmax(deep_model.predict(X_test_scaled), axis = -1)
predicted_labels = label_encoder.inverse_transform(prediction)
print(f"Predicted Labels: {predicted_labels[:5]}")
print(f"Actual Labels: {list(y_test[:5])}")

Predicted Labels: ['Burlington' 'Cumberland' 'Union' 'Camden' 'Somerset']
Actual Labels: ['Burlington', 'Cumberland', 'Union', 'Camden', 'Morris']


In [11]:
# Save the model
deep_model.save("../Models/NJ_deep_learning.h5")

In [12]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [14]:
confusion_matrix(y_test, predicted_labels)

array([[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,